In [1]:
# load module and pre generate data
from graph_tree_module import Node, sim, graph_tree

import numpy as np

import pickle

# Open the file in binary read mode and load the object
with open("graph_tree.pkl", "rb") as file:
    gt = pickle.load(file)


In [2]:
sliders_data = [{"id": gt.attr_name[i], "min": gt.X_maxmin[1][i], "max": gt.X_maxmin[0][i], "value": gt.root.center[i], "type": "f"} for i in range(len(gt.attr_name))]
# print(sliders_data)

recom_data = [{"i": i, "id": gt.attr_name[i], "max_val": gt.recom[i][0].center[i], "min_val": gt.recom[i][1].center[i]} for i in range(len(gt.attr_name))]
# print(recom_data)

status = gt.get_suround_cluster(gt.root, gt.root.center)
print(status)
def get_suround_node_data(status):
    cur_node = status["cur_cluster"]
    cur_cluster = [{"id": cur_node.id, "position": "center", "x": 300, "y": 200, "color": "red"}]

    contra_node = status["contra_cluster"]
    contra_cluster = [{"id": contra_node[i].id, "position": "left", "x": 100, "y": 150+i*100, "color": "green"} for i in range(len(contra_node))]

    super_node = status["super_cluster"]
    super_cluster = [{"id": super_node.id, "position": "top", "x": 300, "y": 100, "color": "dark red"}] if super_node else []

    sim_node = status["neibour_cluster"]
    sim_cluster = [{"id": sim_node[i].id, "position": "right", "x": 500, "y": 200+i*100, "color": "blue"} for i in range(len(sim_node))] if sim_node else []

    sub_node = status["sub_cluster"]
    sub_cluster = [{"id": sub_node[i].id, "position": "bottom", "x": 200+i*100, "y": 350, "color": "pink"} for i in range(len(sub_node))] if sub_node else []

    return cur_cluster + contra_cluster + super_cluster + sim_cluster + sub_cluster

graph_data = get_suround_node_data(status)

print(graph_data)

def get_tree_data(status):
    cur_node = status["cur_cluster"]
    up1_node = cur_node.up_lv if cur_node.up_lv else []
    up2_node = up1_node.up_lv if up1_node.up_lv else []

tree_data = {
    'name': 'Root',
    'children': [
        {'name': 'Child 1', 'children': [
            {'name': 'Grandchild 1'},
            {'name': 'Grandchild 2'}
        ]},
        {'name': 'Child 2', 'children': [
            {'name': 'Grandchild 3'}
        ]}
    ]
}

[0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ] [0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ]
[0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ] [0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ]
[0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ] [0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ]
[0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.250018

In [3]:
from flask import Flask, render_template, request, jsonify

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

# slider ####################################################################
@app.route('/sliders')
def sliders():
    return jsonify(sliders_data)

@app.route('/handle_slider', methods=['POST'])
def submit_slider_values():
    data = request.json
    print('Received slider values:', data)
    query = np.array([float(val) for val in data.values()])
    cur = gt.search(query)
    print(cur.id)
    print(gt.status)
    global graph_data
    graph_data = get_suround_node_data(gt.status)
    # Process the data as needed
    return jsonify({'status': 'slider respon success', 'received': data})
# slider end ################################################################

# recom ####################################################################
@app.route('/recom')
def recom():
    return jsonify(recom_data)

@app.route('/handle_recom', methods=['POST'])
def handle_recom():
    data = request.json
    button_id = int(data.get('buttonId'))
    val = int(data.get('val'))
    print(button_id, val)
    moved_node = gt.recom[button_id][val]
    gt.get_suround_cluster(moved_node)
    global graph_data
    graph_data = get_suround_node_data(gt.status)
    # Handle the button click based on the button_id
    return jsonify({'status': 'recom respon success', 'buttonId': button_id, 'val': val})
# recom end ####################################################################

@app.route('/graph-data')
def graph():
    return jsonify(graph_data)

@app.route('/tree-data')
def tree():
    return jsonify(tree_data)

if __name__ == '__main__':
    app.run(port=8123)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8123
Press CTRL+C to quit
127.0.0.1 - - [07/Sep/2024 01:09:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2024 01:09:50] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [07/Sep/2024 01:09:50] "GET /static/scripts.js HTTP/1.1" 304 -
127.0.0.1 - - [07/Sep/2024 01:09:50] "GET /sliders HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2024 01:09:50] "GET /recom HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2024 01:09:50] "GET /graph-data HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2024 01:09:51] "GET /tree-data HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2024 01:10:15] "POST /handle_slider HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2024 01:10:15] "GET /graph-data HTTP/1.1" 200 -


Received slider values: {'Popularityslider': '62.03', 'danceabilityslider': '0.7399', 'energyslider': '0.79301', 'keyslider': '5.26', 'loudnessslider': '-24.644', 'modeslider': '0.28', 'speechinessslider': '0.6425', 'acousticnessslider': '0.71', 'instrumentalnessslider': '0.7', 'livenessslider': '0.6336', 'valenceslider': '0.7659', 'temposlider': '132.682', 'duration_in min/msslider': '644977.90165', 'time_signatureslider': '3.19'}
[0.62656566 0.73744713 0.79397783 0.47818182 0.30796667 0.28
 0.6779661  0.71285141 0.70281124 0.63368765 0.77155667 0.54160553
 0.45663717 0.5475    ] [0.45601256 0.5560581  0.76175629 0.55522546 0.79340487 0.
 0.07903395 0.09519779 0.10334684 0.18899144 0.52434241 0.49637021
 0.15762826 0.73965142]
[0.62656566 0.73744713 0.79397783 0.47818182 0.30796667 0.28
 0.6779661  0.71285141 0.70281124 0.63368765 0.77155667 0.54160553
 0.45663717 0.5475    ] [ 4.37091133e-01  5.40213790e-01  6.74770868e-01  5.43188114e-01
  7.59257990e-01 -2.55351296e-15  7.01391100e

127.0.0.1 - - [07/Sep/2024 01:11:25] "POST /handle_slider HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2024 01:11:25] "GET /graph-data HTTP/1.1" 200 -


Received slider values: {'Popularityslider': '62.03', 'danceabilityslider': '0.7399', 'energyslider': '0.79301', 'keyslider': '5.26', 'loudnessslider': '-24.644', 'modeslider': '0.28', 'speechinessslider': '0.6425', 'acousticnessslider': '0.71', 'instrumentalnessslider': '0.7', 'livenessslider': '0.6336', 'valenceslider': '0.7659', 'temposlider': '132.682', 'duration_in min/msslider': '644977.90165', 'time_signatureslider': '3.19'}
[0.62656566 0.73744713 0.79397783 0.47818182 0.30796667 0.28
 0.6779661  0.71285141 0.70281124 0.63368765 0.77155667 0.54160553
 0.45663717 0.5475    ] [0.45601256 0.5560581  0.76175629 0.55522546 0.79340487 0.
 0.07903395 0.09519779 0.10334684 0.18899144 0.52434241 0.49637021
 0.15762826 0.73965142]
[0.62656566 0.73744713 0.79397783 0.47818182 0.30796667 0.28
 0.6779661  0.71285141 0.70281124 0.63368765 0.77155667 0.54160553
 0.45663717 0.5475    ] [ 4.37091133e-01  5.40213790e-01  6.74770868e-01  5.43188114e-01
  7.59257990e-01 -2.55351296e-15  7.01391100e

...

2 1
[0.37626263 0.45642851 0.25763758 0.14876033 0.64909687 1.
 0.01773324 0.85431499 0.01902023 0.12227291 0.27099151 0.4585729
 0.09641026 0.67329545] [0.4101459  0.49256889 0.36862392 0.16727273 0.68366886 1.
 0.03219391 0.6608523  0.02663339 0.14847234 0.29189958 0.45699643
 0.11275586 0.68333333]
[0.37626263 0.45642851 0.25763758 0.14876033 0.64909687 1.
 0.01773324 0.85431499 0.01902023 0.12227291 0.27099151 0.4585729
 0.09641026 0.67329545] [0.38355465 0.53743531 0.40269881 0.19203269 0.68464131 1.
 0.0448142  0.68572887 0.02580899 0.1602539  0.43829216 0.44910447
 0.10385525 0.6994382 ]
[0.37626263 0.45642851 0.25763758 0.14876033 0.64909687 1.
 0.01773324 0.85431499 0.01902023 0.12227291 0.27099151 0.4585729
 0.09641026 0.67329545] [0.4101459  0.49256889 0.36862392 0.16727273 0.68366886 1.
 0.03219391 0.6608523  0.02663339 0.14847234 0.29189958 0.45699643
 0.11275586 0.68333333]
[0.37626263 0.45642851 0.25763758 0.14876033 0.64909687 1.
 0.01773324 0.85431499 0.01902023 0.1222

127.0.0.1 - - [07/Sep/2024 01:19:55] "POST /handle_recom HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2024 01:19:55] "GET /graph-data HTTP/1.1" 200 -


6 1
[0.37626263 0.45642851 0.25763758 0.14876033 0.64909687 1.
 0.01773324 0.85431499 0.01902023 0.12227291 0.27099151 0.4585729
 0.09641026 0.67329545] [0.4101459  0.49256889 0.36862392 0.16727273 0.68366886 1.
 0.03219391 0.6608523  0.02663339 0.14847234 0.29189958 0.45699643
 0.11275586 0.68333333]
[0.37626263 0.45642851 0.25763758 0.14876033 0.64909687 1.
 0.01773324 0.85431499 0.01902023 0.12227291 0.27099151 0.4585729
 0.09641026 0.67329545] [0.38355465 0.53743531 0.40269881 0.19203269 0.68464131 1.
 0.0448142  0.68572887 0.02580899 0.1602539  0.43829216 0.44910447
 0.10385525 0.6994382 ]
[0.37626263 0.45642851 0.25763758 0.14876033 0.64909687 1.
 0.01773324 0.85431499 0.01902023 0.12227291 0.27099151 0.4585729
 0.09641026 0.67329545] [0.4101459  0.49256889 0.36862392 0.16727273 0.68366886 1.
 0.03219391 0.6608523  0.02663339 0.14847234 0.29189958 0.45699643
 0.11275586 0.68333333]
[0.37626263 0.45642851 0.25763758 0.14876033 0.64909687 1.
 0.01773324 0.85431499 0.01902023 0.1222

127.0.0.1 - - [07/Sep/2024 01:19:57] "POST /handle_recom HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2024 01:19:57] "GET /graph-data HTTP/1.1" 200 -


12 1
[0.35988154 0.41785169 0.28005191 0.38981437 0.5317023  1.
 0.02936231 0.82096974 0.81417818 0.13158678 0.27485574 0.41608326
 0.07890874 0.68979058] [0.38736227 0.50479562 0.36914544 0.42630607 0.64827699 1.
 0.04072134 0.72824872 0.20825747 0.14925412 0.39111376 0.4408227
 0.09615436 0.69862605]
[0.35988154 0.41785169 0.28005191 0.38981437 0.5317023  1.
 0.02936231 0.82096974 0.81417818 0.13158678 0.27485574 0.41608326
 0.07890874 0.68979058] [0.44017957 0.51780136 0.65391656 0.44026177 0.75100433 1.
 0.05753974 0.25911434 0.13463644 0.18605759 0.47602233 0.48493261
 0.13730071 0.72769953]
[0.35988154 0.41785169 0.28005191 0.38981437 0.5317023  1.
 0.02936231 0.82096974 0.81417818 0.13158678 0.27485574 0.41608326
 0.07890874 0.68979058] [0.38736227 0.50479562 0.36914544 0.42630607 0.64827699 1.
 0.04072134 0.72824872 0.20825747 0.14925412 0.39111376 0.4408227
 0.09615436 0.69862605]
[0.35988154 0.41785169 0.28005191 0.38981437 0.5317023  1.
 0.02936231 0.82096974 0.81417818 0.13